In [1]:
import jeans 
import numpy as np

In [ ]:
import jeans 
import numpy as np

M200 = 1e12 # virial mass, Msun
R200 = 200 # virial radius, kpc
c = 10 # concentration, dimensionless
q0 = 0.8
rm = 10
GN = 4.302e-6 # gravitational constant in kpc (km/s)^2 Msun^-1

# Example baryonic potential: Miyamoto-Nagai disk
def Phi_b(r, th):
    Md = 6.4e10
    a = 3.5
    b = 0.53
    return -GN*Md/np.sqrt(r**2 * np.sin(th)**2 + (a + np.sqrt(b**2 + r**2 * np.cos(th)**2))**2)


profile = jeans.squashed(rm, M200, c, alpha=0.2, q0=q0, Phi_b=Phi_b)

if profile:
    print("Profile generated successfully")

ValueError: not enough values to unpack (expected 3, got 2)

In [3]:
profile.outer.halo_type

'NFW'

In [10]:
def test(a, b=1, c=1):
    print(f'a:{a}, b:{b}, c:{c}')
    return a + b + c

test(1,3,2)

a:1, b:3, c:2


6

In [16]:
import datetime as dt 

now = dt.datetime.now()
print(now.strftime("%Y_%m_%d_%H_%M_%S"))

2025_09_24_14_48_54


In [18]:
check = 1.2

if check:
    print("Check is True")

Check is True


In [6]:
profile.outer.halo_type

'NFW'

In [7]:
filename = "example_profile.npz"
profile.save(filename)

In [8]:
from jeans.classes import load_profile

In [9]:
new_profile = load_profile(filename)

In [10]:
new_profile.rho_sph_avg(10)

np.float64(5843565.990419926)

In [11]:
alpha = 0.17

profile = jeans.squashed(rm, M200, c, alpha, q0=q_sph, Phi_b=Phi_b)

if profile:
    print("Profile generated successfully")

Profile generated successfully


In [12]:
profile.outer.halo_type

'Einasto'